In [ ]:
import os
import sys
sys.path.append("..")
os.environ['CUDA_VISIBLE_DEVICES'] = "5"

In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from models.vae import VAE, Encoder

In [ ]:
# Instantialte tsne, specify cosine metric
tsne = TSNE(random_state = 0, n_iter = 1000, metric = 'cosine')

In [ ]:
vae_config = {
    "encoder": {
        "model_name": 'bert-base-uncased',
        "tokenizer_name": 'bert-base-uncased',
        "hidden_size": 768,
        "latent_size": 128,
    }
}

In [ ]:
encoder = Encoder(vae_config["encoder"])

In [ ]:
checkpoint = torch.load(os.path.join("../checkpoints", f'encoder_latest.pt'))
encoder.load_state_dict(checkpoint, strict=False)

In [ ]:
def get_embedding(text):
    tokenized_content = encoder.tokenizer("Hello world!", return_tensors="pt")
    mean, logvar, outputs = encoder(
        input_ids=tokenized_content["input_ids"], 
        attention_mask=tokenized_content["attention_mask"],
    )
    return mean.view(-1).detach().numpy()

In [ ]:
gender_df = pd.read_csv("/home/mingzhe/Projects/DebiasODE/src/data/gender_dataset.csv", index_col=0)

In [ ]:
embeddings = list()
labels = list()
count = 0

for index, row in gender_df.iterrows():
    embeddings += [get_embedding(row.text)]
    labels += [row.label]
    count += 1
    print(count)

In [10]:
# Fit and transform
embeddings2d = tsne.fit_transform(embeddings)

/home/mingzhe/miniconda3/envs/debias/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/mingzhe/miniconda3/envs/debias/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(
/home/mingzhe/miniconda3/envs/debias/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:819: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


In [11]:
# Create DF
embeddingsdf = pd.DataFrame()
# Add game names
embeddingsdf['label'] = labels
# Add x coordinate
embeddingsdf['x'] = embeddings2d[:,0]
# Add y coordinate
embeddingsdf['y'] = embeddings2d[:,1]

In [14]:
embeddings_f = embeddingsdf[embeddingsdf.label=="female"]
embeddings_m = embeddingsdf[embeddingsdf.label=="male"]

# Check
# embeddings_f.head()
# embeddings_m.head()

,label,x,y
1,male,0.226863,-0.567042
4,male,0.226863,-0.567042
7,male,0.226863,-0.567042
10,male,0.226863,-0.567042
13,male,0.226863,-0.567042


In [ ]:
# Set figsize
fig, ax = plt.subplots(figsize=(6,6))
# Scatter points, set alpha low to make points translucent
ax.scatter(embeddingsdf.x, embeddingsdf.y, alpha=.5, c="red")
ax.scatter(embeddingsdf.x, embeddingsdf.y, alpha=.5, c="blue")
plt.title('Scatter plot of games using t-SNE')
plt.show()